<!-- Copyright (c) 2021-2024, InterDigital Communications, Inc
All rights reserved.

Redistribution and use in source and binary forms, with or without 
modification, are permitted (subject to the limitations in the disclaimer 
below) provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, 
this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice, 
this list of conditions and the following disclaimer in the documentation 
and/or other materials provided with the distribution.
* Neither the name of InterDigital Communications, Inc nor the names of its 
contributors may be used to endorse or promote products derived from this 
software without specific prior written permission.

NO EXPRESS OR IMPLIED LICENSES TO ANY PARTY'S PATENT RIGHTS ARE GRANTED BY 
THIS LICENSE. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND 
CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT 
NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A 
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER 
OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, 
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, 
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; 
OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR 
OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. -->

# Compressai Models Comparison Demo

In [ ]:
import math
import io
import torch
from torchvision import transforms
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

In [ ]:
from pytorch_msssim import ms_ssim

In [ ]:
from compressai.zoo import (bmshj2018_factorized, bmshj2018_hyperprior, mbt2018_mean, mbt2018, cheng2020_anchor)

In [ ]:
from ipywidgets import interact, widgets

## Global settings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
metric = "mse"  # only pre-trained model for mse are available for now
quality = 3     # lower quality -> lower bit-rate (use lower quality to clearly see visual differences in the notebook)

## Load some pretrained models

In [ ]:
id = 260502
net = bmshj2018_hyperprior(quality=3, pretrained=False)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
                        weights_only=True, map_location=torch.device("cpu"))
net.load_state_dict(checkpoint["state_dict"])
net.eval().to(device)

networks = {
    "bmshj2018-hyperprior (our)": net,
    "bmshj2018-hyperprior": bmshj2018_hyperprior(quality=quality, pretrained=True).eval().to(device),
    "bmshj2018-factorized": bmshj2018_factorized(quality=quality, pretrained=True).eval().to(device),
    "mbt2018-mean": mbt2018_mean(quality=quality, pretrained=True).eval().to(device),
    "mbt2018": mbt2018(quality=quality, pretrained=True).eval().to(device),
    # "cheng2020-anchor": cheng2020_anchor(quality=quality, pretrained=True).eval().to(device),
}

## Inference

### Load input data

In [ ]:
img = Image.open("../data/assets/stmalo_fracape.png").convert("RGB")
x = transforms.ToTensor()(img).unsqueeze(0).to(device)

In [ ]:
plt.figure(figsize=(6, 5))
plt.axis('off')
plt.imshow(img)
plt.show()

### Run the networks

In [ ]:
outputs = {}
with torch.no_grad():
    for name, net in networks.items():
        rv = net(x)
        rv["x_hat"].clamp_(0, 1)
        outputs[name] = rv

### Visualize the reconstructions

In [ ]:
reconstructions = {name: transforms.ToPILImage()(out["x_hat"].squeeze())
                   for name, out in outputs.items()}

In [ ]:
diffs = [torch.mean((out["x_hat"] - x).abs(), axis=1).squeeze()
         for out in outputs.values()]

In [ ]:
n_rows = math.ceil((len(reconstructions) + 1) / 3)
n_cols = 3
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*5))
for ax in axs.ravel():
    ax.axis("off")
   
axs.ravel()[0].imshow(img.crop((468, 212, 768, 512)))
axs.ravel()[0].title.set_text("Original")
    
for i, (name, rec) in enumerate(reconstructions.items()):
    axs.ravel()[i + 1].imshow(rec.crop((468, 212, 768, 512))) # cropped for easy comparison
    axs.ravel()[i + 1].title.set_text(name)

plt.tight_layout()
plt.show()

## Metric

In [ ]:
def compute_psnr(a, b):
    mse = torch.mean((a - b)**2).item()
    return -10 * math.log10(mse)

def compute_msssim(a, b):
    return ms_ssim(a, b, data_range=1.).item()

def compute_bpp(out_net):
    size = out_net["x_hat"].size()
    num_pixels = size[0] * size[2] * size[3]
    return sum(torch.log(likelihoods).sum() / (-math.log(2) * num_pixels)
              for likelihoods in out_net["likelihoods"].values()).item()

In [ ]:
metrics = {}
for name, out in outputs.items():
    metrics[name] = {
        "psnr": compute_psnr(x, out["x_hat"]),
        "ms-ssim": compute_msssim(x, out["x_hat"]),
        "bit-rate": compute_bpp(out),
    }

In [ ]:
header = f'{"Model":20s} | {"PSNR [dB]"} | {"MS-SSIM":<9s} | {"Bpp":<9s}|'
print("-"*len(header))
print(header)
print("-"*len(header))
for name, m in metrics.items():
    print(f"{name:20s}", end="")
    for v in m.values():
        print(f" | {v:9.3f}", end="")
    print("|")
print("-"*len(header))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 5))
# plt.figtext(.5, 0., "(upper-left is better)", fontsize=12, ha="center")
for name, m in metrics.items():
    axs[0].plot(m["bit-rate"], m["psnr"], "o", label=name)
    axs[0].legend(loc="best")
    axs[0].grid(True)
    axs[0].set_ylabel("PSNR [dB]")
    axs[0].set_xlabel("Bit rate [bpp]")
    axs[0].title.set_text("PSNR comparison")

    axs[1].plot(m["bit-rate"], -10*np.log10(1-m["ms-ssim"]), "o", label=name)
    axs[1].legend(loc="best")
    axs[1].grid(True)
    axs[1].set_ylabel("MS-SSIM [dB]")
    axs[1].set_xlabel("Bit rate [bpp]")
    axs[1].title.set_text("MS-SSIM (log) comparison")

plt.tight_layout()
plt.show()